<a href="https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import numpy as np
import pandas as pd
import itertools
import warnings
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, LarsCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
import scipy.stats
import statsmodels.api as sm
import statsmodels.tools.eval_measures as bias
import seaborn as sns
warnings.filterwarnings("ignore")

In [54]:
import numpy as np
from sklearn.linear_model import LinearRegression

np.set_printoptions(precision = 1)

n = 50
# assume these are the original features
x1 = np.random.randint(size = n, low = 50, high = 100)
x2 = np.random.randint(size = n, low = 2, high = 20)
x3 = np.random.randint(size = n, low = 5, high = 50)

# and this is the model with some gaussian noise
y = 12 + x1 + 4 * x2 - 5 * x3 \
    + 3 * x1**2 \
    - 2 * np.sqrt(x2) \
    + 4 * np.log(x1) \
    + 2 * x1 * x3 \
    + np.random.normal(size = n, loc = 0, scale = 0.05)

# first do regression as such
X = np.column_stack((x1, x2, x3))

raw = LinearRegression()
raw.fit(X, y)
print(f'A very bad model: {raw.intercept_:.1f}', np.round(raw.coef_, 1))

# TRANSFORMATIONS
# add squares of raw features
tf1 = np.power(x1, 2)
tf2 = np.power(x2, 2)
tf3 = np.power(x3, 2)
X = np.column_stack((X, tf1, tf2, tf3))
# add roots
tf4 = np.sqrt(x1)
tf5 = np.sqrt(x2)
tf6 = np.sqrt(x3)
X = np.column_stack((X, tf4, tf5, tf6))
# add logarithms
tf7 = np.log(x1)
tf8 = np.log(x2)
tf9 = np.log(x3)
X = np.column_stack((X, tf4, tf5, tf6))
# add products
tf10 = np.multiply(x1, x2)
tf11 = np.multiply(x2, x3)
tf12 = np.multiply(x1, x3)
X = np.column_stack((X, tf10, tf11, tf12))

model = LinearRegression()
model.fit(X, y)
for (desired, obtained) in zip([12, 1, 4, -5, 3, 0, 0, 0, -2, 0, 4, 0, 0, 0, 0, 2],
                                   [round(model.intercept_)] + list([int(i) for i in np.round(model.coef_)])):
    print(desired, obtained, 'ok' if desired == obtained else 'mismatch')
    
print(f'A transformed model: {raw.intercept_:.1f}', np.round(raw.coef_, 1))

A very bad model: -21619.4 [519.6  -2.7 151.4]
12 13 mismatch
1 1 ok
4 4 ok
-5 -5 ok
3 3 ok
0 0 ok
0 0 ok
0 1 mismatch
-2 -1 mismatch
0 0 ok
4 1 mismatch
0 -1 mismatch
0 0 ok
0 0 ok
0 0 ok
2 2 ok
A transformed model: -21619.4 [519.6  -2.7 151.4]


In [55]:
#función que calcula la predicción Y[] de un X[] usando los parametros o coeficientes de regresión beta[]
def pronostica(params,X): #params:parámetros de regresión; X:datos de regresores 
    y = []
    i = 0
    for rows in X:
        y.append(np.matmul(params,X[i]))
        i = i + 1
    return(y)

In [56]:
def dibuja_hist(df,colour,name,Xlabel,Ylabel,title):
    plt.figure()
    df.hist(column=0, bins=25, grid=False, figsize=(6,3), color=colour, zorder=2, rwidth=0.9)
    plt.xticks(rotation=90)
    plt.xlabel(Xlabel)
    plt.ylabel(Ylabel)
    #https://stackoverflow.com/questions/9662995/matplotlib-change-title-and-colorbar-text-and-tick-colors/42020486
    mytitle = plt.title(title) # get the title property handler   #plt.getp(title_obj)  
    plt.setp(mytitle, color='#ff8000')                            # set the color of title
    myax = plt.axes()   # get the axis property handler           # plt.getp(myax) print its propieties
    myax.xaxis.label.set_color('#ff8000')
    myax.yaxis.label.set_color('#ff8000')
    myax.tick_params(colors='#ff8000', which='both')              # myax.spines['bottom'].set_color('yellow')
    plt.savefig(name, transparent=True)
    plt.show()                                                    # o plt.save_en_algún_formato()

## Regresión lineal en predicción de demanda eléctrica

A continuación aplicaremos técnicas de expansión de base aplicados a predicción de demanda eléctrica por regresión, tal como se hizo en el la [tarea 3](https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea3.ipynb#scrollTo=Ext-myaET2n5). Recordando que la variable independiente $X$ serán los datos de demanda del día anterior, y los datos independiente $Y$ serán los datos de días con una mayor correlación. Compararemos el desempeño de usar transformación variables y algunas tpecnicas de reducción de dimensiones como **stepwise**.

Los datos usados en esta sección están disponibles en [demanda.csv](https://drive.google.com/file/d/1KpY2p4bfVEwGRh5tJjMx9QpH6SEwrUwH/view?usp=sharing)

In [57]:
df = pd.read_csv('demanda.csv')

In [58]:
## Procesamos datos de entrenamiento 
df1 = df.loc[df.iloc[:,19].isin(['T'])] ## 'T' = training set
df2 = df1['Y']  
df1 = df1[['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']] ## Regresores

Ahora, cargamos los datos de prueba en los arreglos `X_train` y `y_train`:

In [59]:
## Procesamos datos de prueba 
dft = df.loc[df.iloc[:,19].isin(['F'])] ## 'F' = test set
dft2 = dft['Y']
dft1 = dft[['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']] ## Regresores

Estandarizamos los datos de los regresores `X_train` y `X_test` restando la media y dividiendo entre la varianza.

In [60]:
df1.std(numeric_only = True) 
df1.mean(numeric_only = True)
df1 = df1 - df1.mean(numeric_only = True)
df1 = df1 / df1.std(numeric_only = True) 
X_train = df1.to_numpy()   ## Predictors
X_train = sm.add_constant(X_train)
y_train = df2.to_numpy()   ## Outcome

# TRANSFORMATIONS
# add squares of raw features
tf1 = np.power(x1, 2)
tf2 = np.power(x2, 2)
tf3 = np.power(x3, 2)
X_train = np.column_stack((X_train, tf1, tf2, tf3))
# add roots
tf4 = np.sqrt(x1)
tf5 = np.sqrt(x2)
tf6 = np.sqrt(x3)
X_train = np.column_stack((X_train, tf4, tf5, tf6))
# add logarithms
tf7 = np.log(x1)
tf8 = np.log(x2)
tf9 = np.log(x3)
X_train = np.column_stack((X_train, tf4, tf5, tf6))
# add products
tf10 = np.multiply(x1, x2)
tf11 = np.multiply(x2, x3)
tf12 = np.multiply(x1, x3)
X_train = np.column_stack((X_train, tf10, tf11, tf12))

ValueError: ignored

In [ ]:
dft1.std(numeric_only = True) 
dft1.mean(numeric_only = True)
dft1 = dft1 - dft1.mean(numeric_only = True)
dft1 = dft1 / dft1.std(numeric_only = True) 
X_test = dft1.to_numpy()   ## Predictors
X_test = sm.add_constant(X_test)
y_test = dft2.to_numpy()   ## Outcome

# TRANSFORMATIONS
# add squares of raw features
tf1 = np.power(x1, 2)
tf2 = np.power(x2, 2)
tf3 = np.power(x3, 2)
X = np.column_stack((X, tf1, tf2, tf3))
# add roots
tf4 = np.sqrt(x1)
tf5 = np.sqrt(x2)
tf6 = np.sqrt(x3)
X = np.column_stack((X, tf4, tf5, tf6))
# add logarithms
tf7 = np.log(x1)
tf8 = np.log(x2)
tf9 = np.log(x3)
X = np.column_stack((X, tf4, tf5, tf6))
# add products
tf10 = np.multiply(x1, x2)
tf11 = np.multiply(x2, x3)
tf12 = np.multiply(x1, x3)
X = np.column_stack((X, tf10, tf11, tf12))

A continuación, obtenemos un modelo de predicción de los datos de entrenamiento usando regresión lineal.

In [ ]:
model   = sm.OLS(y_train, X_train)
results = model.fit()
#print(results.summary()) 
#print(results.params)

Ahora, calculamos los errores entre la predicción `y_pred` y los datos de entrenamiento `y_train`. Los errores son representados por un histograma.

In [ ]:
y_pred     = pronostica(results.params,X_train)
error      = y_train - y_pred
err_train  = mean_absolute_error(y_pred,y_train)
bias_train = bias.bias(y_pred,y_train, axis=0)
df         = pd.DataFrame(error,y_train)

In [ ]:
dibuja_hist(df,colour='#777bd4',name='hist11.png',Xlabel="Error",Ylabel="Frecuencia",title="Errores de predicción de demanda eléctrica (entrenamiento)")

Ahora, utilizamos el modelo obtenido con los datos de entrenamiento para predecir los datos de prueba. Además,  calculamos los errores entre la predicción `y_pred2` y los datos de prueba $Yt$. Los errores de la predicción con datos de prueba son representados por un histograma.

In [ ]:
y_pred2   = pronostica(results.params,X_test) #print(y_pred2)
error2    = y_test - y_pred2
err_test  = mean_absolute_error(y_test,y_pred2)
bias_test = bias.bias(y_test,y_pred2,axis=0)
df        = pd.DataFrame(error2,y_test)

In [ ]:
dibuja_hist(df,colour='#76ced6',name='hist12.png',Xlabel="Error",Ylabel="Frecuencia",title="Errores de predicción de demanda eléctrica (prueba)")

Ahora, comparamos el **error absoluto medio (MAE)** y **bias** de los datos de entrenamiento así como de los datos de prueba en la predicción de demanda eléctrica.

In [ ]:
print("MAE y bias del modelo de regresión con datos de entrenamiento:", err_train, "," , bias_train)
print("MAE y bias del modelo de regresión con datos de prueba:" , err_test, "," , bias_test) 